In [ ]:
import requests
import time
import re
import pandas as pd
from bs4 import BeautifulSoup,  NavigableString, Tag
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from csv import writer
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

In [ ]:
options = Options()
ua = UserAgent()
userAgent = ua.random

# LISTING SCRAPING

In [ ]:
# Listing urls

first_url = 'https://rentals.ca/montreal/1-bedrooms'
base_url = 'https://rentals.ca/montreal/1-bedrooms?p='
urls = [first_url]
number_of_pages = 82

for x in range(2,number_of_pages):
    url = base_url + str(x)
    urls.append(url)

In [ ]:
# Listing scraping

with open('montreal_housing6.csv', 'w', encoding='utf8', newline='') as f:
    csv_writer = writer(f)
    header = ['Listing_ID', 'Location', 'Plan_info', 'Utilities_info', 'Property_type', 'Property_sub_type','Parking_type', 
              'Parking_spot', 'Lease_term', 'Short_term', 'Furnished', 'Year_built']
    csv_writer.writerow(header)
    
    counter = 0
    for url in urls:
        
        print('Counter:', counter)
        options.add_argument(f'user-agent={userAgent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path='/opt/homebrew/bin/chromedriver')
        driver.get(url)
        driver.implicitly_wait(5)
        
        # soup
        soup = BeautifulSoup(driver.page_source, features='html.parser')
        driver.quit()
        
        main = soup.find('div', attrs={'class':"page-search-results__content"})
        prop_ids = main.find_all('div', attrs={'class':'listing-card-container col-12'})
    
        for item in prop_ids:

            # Property id
            id_ = item['data-property-id']

            # Property address
            location = item.find('h2', class_= "listing-card__title").text.replace('\n','').strip()

            # use id to get property url:
            if url == first_url:
                prop_url = url + '?active-listing-id=' + id_  #for first page

            else:
                prop_url = url + '&active-listing-id=' + id_  #for the rest

            # get property page and soup
            prop_driver = webdriver.Chrome(chrome_options=options, executable_path='/opt/homebrew/bin/chromedriver')
            prop_driver.get(prop_url)
            prop_soup = BeautifulSoup(prop_driver.page_source, features='html.parser')
            prop_driver.quit()

            # Get price, baths, area and availability of first 1 bedroom plan
            plan = prop_soup.find_all('div', attrs={'id':'floor-plan-group1','class':"floor-plan-group__content"})
            if len(plan) == 0:
                plan_info = None
            else:
                plan_info = plan[0].get_text()

            # Get utilities included
            utilities = prop_soup.find_all('li', class_="label--secondary")
            utils = [x.get_text().strip() for x in utilities]
             
            # Get additional features 
            
            about = prop_soup.find_all('li', attrs={'class':"listing-highlighted-info__item"})
            add_info = []
            
            for sub in about:
                if sub is None:
                    info = None
                else:
                    info = sub.p.text
                    
                add_info.append(info)

            all_info = [id_, location, plan_info, utils] + add_info

            csv_writer.writerow(all_info)
            
        counter += 1
            

        
        

# POSTAL CODE SCRAPING FROM GOOGLE MAPS

In [ ]:
post_locations = list(new_df['Location'])
base_url = 'https://maps.google.com/?q='

In [ ]:
with open('postal_codes_dup.csv', 'w', encoding='utf8', newline='') as p:
    to_write = writer(p)
    header = ['Location', 'Postal_code']
    to_write.writerow(header)
    counter = 0

    for loc in post_locations:
        url = base_url + loc
        options.add_argument(f'user-agent={userAgent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path='/opt/homebrew/bin/chromedriver')
        driver.get(url)
        driver.implicitly_wait(5)

        soup = BeautifulSoup(driver.page_source, features='html.parser')
        driver.quit()

        data = soup.find('div', attrs={'class':"Dn7b2c g9poTc", 'id':"Xq6CCb"}) 
        
        conts = data.contents

        if conts is None or len(conts) < 1:
            ps_code = None
            
        else:

            text = str(conts[0])
            #ps_code = re.search('H[0-9][A-Z]\%20[0-9][A-Z][0-9]', text)[0].replace('%20',' ')

            # considering both H and J postal codes
            h_ps_code = re.search('H[0-9][A-Z]\%20[0-9][A-Z][0-9]', text)
            j_ps_code = re.search('J[0-9][A-Z]\%20[0-9][A-Z][0-9]', text)
            hj = (h_ps_code, j_ps_code) #(None, postal_code)
            if hj == (None, None):
                ps_code = None
            else:
                temp_ps = [x for x in hj if x is not None][0]
                ps_code = temp_ps[0].replace('%20',' ')

      

    
        info = [loc, ps_code]
        counter += 1
        print(counter, loc, ps_code, '\n')

        to_write.writerow(info)
                